# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [3]:
orders.shape

(397924, 14)

In [4]:
orders['CustomerID'].value_counts()

17841    7847
14911    5677
14096    5111
12748    4596
14606    2700
         ... 
15313       1
17846       1
13185       1
16953       1
16737       1
Name: CustomerID, Length: 4339, dtype: int64

In [5]:
orders['Country'].value_counts()

United Kingdom          354345
Germany                   9042
France                    8342
EIRE                      7238
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1842
Portugal                  1462
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     248
Unspecified                244
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon                     45
Lithuani

In [6]:
#I wanted to get an idea of how many customers we have, number of transaction and country list.

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [7]:
# first aggregate the amount_spent per customer.

In [8]:
cust_group = orders.groupby(['CustomerID']).agg({'amount_spent':sum, 'Country':max}).round(2)
cust_group

,amount_spent,Country
CustomerID,,
12346,77183.60,United Kingdom
12347,4310.00,Iceland
12348,1797.24,Finland
12349,1757.55,Italy
12350,334.40,Norway
...,...,...
18280,180.60,United Kingdom
18281,80.82,United Kingdom
18282,178.05,United Kingdom


In [9]:
#then I create 20 bins to separate customers in 0-5 percentile increments. 

In [10]:
labels = ['0','5','10','15','20','25','30','35','40','45','50','55','60','65','70','75','80','85','90','95']

In [11]:
cust_group['Percentile'] = pd.qcut(cust_group['amount_spent'],20, labels = labels)
cust_group

,amount_spent,Country,Percentile
CustomerID,,,
12346,77183.60,United Kingdom,95
12347,4310.00,Iceland,90
12348,1797.24,Finland,75
12349,1757.55,Italy,75
12350,334.40,Norway,25
...,...,...,...
18280,180.60,United Kingdom,10
18281,80.82,United Kingdom,0
18282,178.05,United Kingdom,10


In [12]:
#Now label the VIP and Preferred customers. 

In [13]:
def status (x): 
    if x == '95':
        return 'VIP'
    elif x == '90':
        return 'PreferredCustomer'
    elif x == '85':
        return 'PreferredCustomer'
    elif x == '80':
        return 'PreferredCustomer'
    elif x == '75':
        return 'PreferredCustomer'
    else:
        return 'No status'

In [14]:
cust_group["Status"]=cust_group['Percentile'].apply(status)
cust_group

,amount_spent,Country,Percentile,Status
CustomerID,,,,
12346,77183.60,United Kingdom,95,VIP
12347,4310.00,Iceland,90,PreferredCustomer
12348,1797.24,Finland,75,PreferredCustomer
12349,1757.55,Italy,75,PreferredCustomer
12350,334.40,Norway,25,No status
...,...,...,...,...
18280,180.60,United Kingdom,10,No status
18281,80.82,United Kingdom,0,No status
18282,178.05,United Kingdom,10,No status


In [15]:
#we now have a flag identifying the VIP and Preferred Customers, easy to select this group. 

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [16]:
#can't make a groupby based on the one flag, need to split it into different columns. 

In [17]:
def VIP (x): 
    if x == '95':
        return True
    else:
        return False
    
def PrefCust (x):
    if x == '90':
        return True
    elif x == '85':
        return True
    elif x == '80':
        return True
    elif x == '75':
        return True
    else:
        return False

In [18]:
cust_group["VIP"]=cust_group['Percentile'].apply(VIP)
cust_group['PreferredCustomer']=cust_group['Percentile'].apply(PrefCust)
cust_group

,amount_spent,Country,Percentile,Status,VIP,PreferredCustomer
CustomerID,,,,,,
12346,77183.60,United Kingdom,95,VIP,True,False
12347,4310.00,Iceland,90,PreferredCustomer,False,True
12348,1797.24,Finland,75,PreferredCustomer,False,True
12349,1757.55,Italy,75,PreferredCustomer,False,True
12350,334.40,Norway,25,No status,False,False
...,...,...,...,...,...,...
18280,180.60,United Kingdom,10,No status,False,False
18281,80.82,United Kingdom,0,No status,False,False
18282,178.05,United Kingdom,10,No status,False,False


In [19]:
Country_pivot = cust_group.pivot_table(index= ['Country'], values = ['VIP', 'PreferredCustomer'], aggfunc = {'VIP':'sum', 'PreferredCustomer':'sum'}).reset_index()
Country_counts = Country_pivot.copy()
Country_counts

,Country,PreferredCustomer,VIP
0,Australia,2,1
1,Austria,1,0
2,Bahrain,0,0
3,Belgium,10,1
4,Brazil,False,False
5,Canada,1,0
6,Channel Islands,3,1
7,Cyprus,1,1
8,Czech Republic,False,False
9,Denmark,2,1


In [20]:
Country_counts.sort_values(by='VIP',ascending=False,inplace=True)
Country_counts

,Country,PreferredCustomer,VIP
35,United Kingdom,755,177
14,Germany,29,10
13,France,20,9
32,Switzerland,6,3
26,Portugal,5,2
19,Japan,2,2
30,Spain,7,2
10,EIRE,1,2
0,Australia,2,1
12,Finland,4,1


In [21]:
#The United Kingdom has the most VIPs. 

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [22]:
#To get the most total customers, I need to add up VIPs and PreferredCustomers (first make them numericals).

In [23]:
Country_counts.dtypes

Country              object
PreferredCustomer    object
VIP                  object
dtype: object

In [24]:
Country_counts['VIP'].replace('False','0', inplace=True)

In [25]:
Country_counts['VIP'].astype(int)

35    177
14     10
13      9
32      3
26      2
19      2
30      2
10      2
0       1
12      1
29      1
31      1
23      1
9       1
7       1
6       1
3       1
24      1
33      0
28      0
27      0
34      0
25      0
18      0
22      0
21      0
20      0
1       0
17      0
16      0
15      0
11      0
8       0
5       0
4       0
2       0
36      0
Name: VIP, dtype: int64

In [26]:
Country_counts['PreferredCustomer'].replace('False','0', inplace=True)

In [27]:
Country_counts['PreferredCustomer'].astype(int)

35    755
14     29
13     20
32      6
26      5
19      2
30      7
10      1
0       2
12      4
29      0
31      1
23      0
9       2
7       1
6       3
3      10
24      6
33      0
28      0
27      0
34      0
25      1
18      5
22      1
21      0
20      1
1       1
17      2
16      1
15      1
11      0
8       0
5       1
4       0
2       0
36      0
Name: PreferredCustomer, dtype: int64

In [28]:
SpecialCust = Country_counts['PreferredCustomer']+Country_counts['VIP'] #forgot the syntax, got it from https://www.kite.com/python/answers/how-to-sum-two-columns-in-a-pandas-dataframe-in-python#:~:text=Use%20the%20addition%20operator%20to%20sum%20two%20columns&text=Create%20a%20new%20column%20in,add%20it%20to%20the%20DataFrame
Country_counts['SpecialCust'] = SpecialCust
Country_counts

,Country,PreferredCustomer,VIP,SpecialCust
35,United Kingdom,755,177,932
14,Germany,29,10,39
13,France,20,9,29
32,Switzerland,6,3,9
26,Portugal,5,2,7
19,Japan,2,2,4
30,Spain,7,2,9
10,EIRE,1,2,3
0,Australia,2,1,3
12,Finland,4,1,5


In [29]:
Country_counts.sort_values(by='SpecialCust',ascending=False,inplace=True)
Country_counts

,Country,PreferredCustomer,VIP,SpecialCust
35,United Kingdom,755,177,932
14,Germany,29,10,39
13,France,20,9,29
3,Belgium,10,1,11
32,Switzerland,6,3,9
30,Spain,7,2,9
26,Portugal,5,2,7
24,Norway,6,1,7
18,Italy,5,0,5
12,Finland,4,1,5


In [30]:
# The UK has the most VIP and Preferred Customers

In [31]:
#I find the question completely different from the question in challenge 2, so don't really understand how the thinking process and workflow can be the same... Just a thought. 